In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from torch_geometric.data import Data
import warnings
warnings.filterwarnings("ignore")
import torch


In [3]:
data=pd.read_csv("data_consolidation_standardization.csv")
data=data.drop(labels="Unnamed: 0", axis=1)
# data_test=data.loc[data['CellID']<=100]

In [4]:
data.sort_values(['CellID','datetime'], inplace=True)
data_test=data

In [5]:
data_test

,datetime,CellID,internet,calls,sms,longitude,latitude
0,2013-11-01 00:00:00,1,-1.045031,-0.739046,-0.846748,9.160012,45.358657
100,2013-11-01 01:00:00,1,-1.074765,-0.750493,-0.865873,9.160012,45.358657
200,2013-11-01 02:00:00,1,-1.087136,-0.754285,-0.873567,9.160012,45.358657
300,2013-11-01 03:00:00,1,-1.096080,-0.757753,-0.875403,9.160012,45.358657
400,2013-11-01 04:00:00,1,-1.109877,-0.756706,-0.878479,9.160012,45.358657
...,...,...,...,...,...,...,...
16399,2013-11-07 19:00:00,100,-0.050505,-0.004918,-0.064944,9.160606,45.568069
16499,2013-11-07 20:00:00,100,-0.141303,-0.239126,-0.156799,9.160606,45.568069
16599,2013-11-07 21:00:00,100,-0.231419,-0.426115,-0.262352,9.160606,45.568069
16699,2013-11-07 22:00:00,100,-0.319378,-0.600432,-0.395659,9.160606,45.568069


# 根据需要选择不同的基站或者流量/语音/短信数据

In [38]:
import pandas as pd

# 加载数据
# data = pd.read_csv('path_to_your_data.csv')  # 根据您的文件路径和文件名进行调整

data = data_test  # 根据您的文件路径和文件名进行调整

# 将时间字符串转换为 pandas 的 datetime 对象，这假设 'datetime' 是存储时间的列
data['datetime'] = pd.to_datetime(data['datetime'])

def extract_time_series(data, cell_id, features):
    """
    提取特定基站的时间序列数据。
    Args:
    data (DataFrame): 包含所有数据的 DataFrame。
    cell_id (int): 基站ID。
    features (list): 要提取的特征名列表，如 ['internet', 'calls', 'sms']。
    
    Returns:
    DataFrame: 包含特定基站和特征的时间序列数据。
    """
    # 过滤特定基站的数据并选择指定的特征
    columns = ['datetime'] + features  # 构建列名列表，包括时间和特征列
    specific_data = data[data['CellID'] == cell_id][columns]
    specific_data.set_index('datetime', inplace=True)
    return specific_data

# 示例：提取基站ID为1的基站的流量和短信数据
cell_id = 1
features = ['internet', 'sms','calls']  # 选择要用到的特征，后面代码会修改哪些使用
ts_data = extract_time_series(data, cell_id, features)
print(ts_data.head())

                     internet       sms     calls
datetime                                         
2013-11-01 00:00:00 -1.045031 -0.846748 -0.739046
2013-11-01 01:00:00 -1.074765 -0.865873 -0.750493
2013-11-01 02:00:00 -1.087136 -0.873567 -0.754285
2013-11-01 03:00:00 -1.096080 -0.875403 -0.757753
2013-11-01 04:00:00 -1.109877 -0.878479 -0.756706


In [39]:
ts_data[features]

,internet,sms,calls
datetime,,,
2013-11-01 00:00:00,-1.045031,-0.846748,-0.739046
2013-11-01 01:00:00,-1.074765,-0.865873,-0.750493
2013-11-01 02:00:00,-1.087136,-0.873567,-0.754285
2013-11-01 03:00:00,-1.096080,-0.875403,-0.757753
2013-11-01 04:00:00,-1.109877,-0.878479,-0.756706
...,...,...,...
2013-11-07 19:00:00,-0.930432,-0.705133,-0.559470
2013-11-07 20:00:00,-0.942378,-0.737554,-0.622987
2013-11-07 21:00:00,-0.963610,-0.743541,-0.673100


In [40]:
ts_data

,internet,sms,calls
datetime,,,
2013-11-01 00:00:00,-1.045031,-0.846748,-0.739046
2013-11-01 01:00:00,-1.074765,-0.865873,-0.750493
2013-11-01 02:00:00,-1.087136,-0.873567,-0.754285
2013-11-01 03:00:00,-1.096080,-0.875403,-0.757753
2013-11-01 04:00:00,-1.109877,-0.878479,-0.756706
...,...,...,...
2013-11-07 19:00:00,-0.930432,-0.705133,-0.559470
2013-11-07 20:00:00,-0.942378,-0.737554,-0.622987
2013-11-07 21:00:00,-0.963610,-0.743541,-0.673100


# 滑动窗口构建数据集和标签

In [44]:
import numpy as np

def create_sequences(data, input_features, output_features, window_size, predict_steps):
    """
    创建滑动窗口数据，允许输入和输出具有不同的特征集。
    Args:
    data (DataFrame): 输入的多特征时间序列数据。
    input_features (list): 用于输入的特征名列表。滑动的时候选择特征去滑
    output_features (list): 用于输出的特征名列表。
    window_size (int): 滑动窗口的大小。
    predict_steps (int): 需要预测的未来步数。
    
    Returns:
    tuple: 包含特征和标签的元组。
    """
    X, y = [], []
    for i in range(len(data) - window_size - predict_steps + 1):
        # 选择输入特征
        X.append(data[input_features].iloc[i:(i + window_size)].values)
        # 选择输出特征
        y.append(data[output_features].iloc[i + window_size:(i + window_size + predict_steps)].values)
    return np.array(X), np.array(y)

# 使用示例
window_size = 6
predict_steps = 3
input_features = ['internet', 'calls']  # 用于输入的特征
output_features = ['sms']  # 预测的特征

# 假设 ts_data 是一个 DataFrame，其中包括多个特征
X, y = create_sequences(ts_data, input_features, output_features, window_size, predict_steps)


In [45]:
X.shape

(160, 6, 2)

In [46]:
y.shape

(160, 3, 1)

# 构建训练集和测试集

In [9]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# 转换数据为Tensor
X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)  # 增加一个维度以适应LSTM输入要求
y_tensor = torch.tensor(y, dtype=torch.float32)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# 创建DataLoaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# 定义LSTM模型并初始化

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, num_output_features):
        """
        初始化 LSTM 模型。
        Args:
        input_dim (int): 每个时间步输入数据的特征数量。
        hidden_dim (int): LSTM 隐藏层的维度。
        num_layers (int): LSTM 层的数量。
        output_dim (int): 需要预测的时间步数。
        num_output_features (int): 输出特征的数量。
        """
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        # 修改全连接层以支持多特征输出
        self.fc = nn.Linear(hidden_dim, output_dim * num_output_features)
        self.output_dim = output_dim
        self.num_output_features = num_output_features
    
    def forward(self, x):
        # LSTM层处理
        _, (hn, _) = self.lstm(x)
        # 将隐藏状态的最后一层通过全连接层
        x = self.fc(hn[-1])
        # 重塑输出维度以匹配预测步数和输出特征数
        return x.view(-1, self.output_dim, self.num_output_features)

# 初始化模型参数
input_dim = 3  # 每个时间步的特征数量，比如同时输入流量、短信和语音
hidden_dim = 50
num_layers = 1
predict_steps = 3  # 预测的时间步数
num_output_features = 2  # 预测的特征数，比如预测流量和短信

# 创建模型实例
model = LSTMModel(input_dim, hidden_dim, num_layers, predict_steps, num_output_features)


In [10]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        
#单特征多步预测，input_dim: 每个时间步输入数据的特征数量。
# hidden_dim: LSTM 隐藏层的维度。
# num_layers: LSTM 层的数量。
# output_dim: 输出层的维度，通常等于你预测的时间步数。

        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])
        return out


input_dim = 1  # 每个时间步只有一个特征，如果同时把流量短信和语音输入，那这里就是3
hidden_dim = 50
num_layers = 1
output_dim = predict_steps  # 预测步数
model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)


# 多特征多步逻辑：初始化模型

# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class LSTMModel(nn.Module):
#     def __init__(self, input_dim, hidden_dim, num_layers, output_dim, num_features):
#         super(LSTMModel, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim * num_features)
#         self.output_dim = output_dim
#         self.num_features = num_features
    
#     def forward(self, x):
#         # LSTM层输出
#         _, (hn, _) = self.lstm(x)
#         # 全连接层调整维度
#         x = self.fc(hn[-1])
#         # 调整输出形状为 [batch_size, output_dim, num_features]
#         return x.view(-1, self.output_dim, self.num_features)

# # 初始化模型
# model = LSTMModel(input_dim=3, hidden_dim=50, num_layers=1, output_dim=3, num_features=2)


# 训练和测试

In [11]:
import torch.optim as optim

# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

def train_model(model, train_loader, criterion, optimizer, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}')

train_model(model, train_loader, criterion, optimizer)




Epoch 1/50, Loss: 0.7121193185448647
Epoch 2/50, Loss: 0.13651218730956316
Epoch 3/50, Loss: 0.0352871201466769
Epoch 4/50, Loss: 0.057912060990929604
Epoch 5/50, Loss: 0.013065560953691602
Epoch 6/50, Loss: 0.01993545237928629
Epoch 7/50, Loss: 0.00653078017057851
Epoch 8/50, Loss: 0.011965732788667083
Epoch 9/50, Loss: 0.004306075978092849
Epoch 10/50, Loss: 0.006642124382779002
Epoch 11/50, Loss: 0.00427630334161222
Epoch 12/50, Loss: 0.004326847498305142
Epoch 13/50, Loss: 0.0033939092536456883
Epoch 14/50, Loss: 0.003427946474403143
Epoch 15/50, Loss: 0.0032992049236781895
Epoch 16/50, Loss: 0.003090238955337554
Epoch 17/50, Loss: 0.003086849581450224
Epoch 18/50, Loss: 0.003100487985648215
Epoch 19/50, Loss: 0.0030308011919260025
Epoch 20/50, Loss: 0.003092305443715304
Epoch 21/50, Loss: 0.0030387986917048693
Epoch 22/50, Loss: 0.003030356310773641
Epoch 23/50, Loss: 0.0030182264163158834
Epoch 24/50, Loss: 0.0030175744905136526
Epoch 25/50, Loss: 0.003025504993274808
Epoch 26/50

In [12]:
def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    print(f'Test MSE: {total_loss/len(test_loader)}')

evaluate_model(model, test_loader, criterion)

Test MSE: 0.0024739420041441917
